1. Veri okuma

Veri setinden gereksiz sütun ('Patient_ID') analiz için gereksiz olduğu için çıkarıldı.

In [7]:
import pandas as pd

file_path = "datasets/thyroid_cancer_risk_data.csv"
df = pd.read_csv(file_path)

df.drop(columns=["Patient_ID"], inplace=True)  # Gereksiz sütunları kaldırma

df.head() 

,Age,Gender,Country,Ethnicity,Family_History,Radiation_Exposure,Iodine_Deficiency,Smoking,Obesity,Diabetes,TSH_Level,T3_Level,T4_Level,Nodule_Size,Thyroid_Cancer_Risk,Diagnosis
0,66,Male,Russia,Caucasian,No,Yes,No,No,No,No,9.37,1.67,6.16,1.08,Low,Benign
1,29,Male,Germany,Hispanic,No,Yes,No,No,No,No,1.83,1.73,10.54,4.05,Low,Benign
2,86,Male,Nigeria,Caucasian,No,No,No,No,No,No,6.26,2.59,10.57,4.61,Low,Benign
3,75,Female,India,Asian,No,No,No,No,No,No,4.10,2.62,11.04,2.46,Medium,Benign
4,35,Female,Germany,African,Yes,Yes,No,No,No,No,9.10,2.11,10.71,2.11,High,Benign


2. Hedef Değişkeni (Thyroid_Cancer_Risk) Sayısala Çevirme

In [8]:
# Hedef değişkeni dönüştürme
risk_mapping = {"Low": 0, "Medium": 1, "High": 2}
df["Thyroid_Cancer_Risk"] = df["Thyroid_Cancer_Risk"].map(risk_mapping)

# Dönüşümü kontrol edelim
print(df["Thyroid_Cancer_Risk"].unique())

[0 1 2]


2.1. Kategorik değişkenleri sayısala dönüştürme

In [9]:
# Yes/No sütunlarını sayısala çevirelim
yes_no_columns = ["Family_History", "Radiation_Exposure", "Iodine_Deficiency", "Smoking", "Obesity", "Diabetes"]

# Yes → 1, No → 0 olarak kodlayalım
df[yes_no_columns] = df[yes_no_columns].applymap(lambda x: 1 if x == "Yes" else 0)

# Dönüşümü kontrol edelim
df[yes_no_columns].head()

/var/folders/dm/hckhrzb5207cp2xy2c6ts38c0000gn/T/ipykernel_32784/3150784243.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[yes_no_columns] = df[yes_no_columns].applymap(lambda x: 1 if x == "Yes" else 0)


,Family_History,Radiation_Exposure,Iodine_Deficiency,Smoking,Obesity,Diabetes
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,1,1,0,0,0,0


2.2. Çoklu Kategorik değişkenleri sayısala dönüştürme

In [10]:
from sklearn.preprocessing import LabelEncoder

# Label Encoding uygulanacak sütunlar
categorical_columns = ["Gender", "Country", "Ethnicity"]

# LabelEncoder kullanarak sayısala çevirelim
encoder = LabelEncoder()
for col in categorical_columns:
    df[col] = encoder.fit_transform(df[col])

# Dönüşümü kontrol edelim
df[categorical_columns].head()

,Gender,Country,Ethnicity
0,1,6,2
1,1,2,3
2,1,5,2
3,0,3,1
4,0,2,0


3. Eksik veri kontrolü

In [11]:
# Eksik veri kontrolü
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)


3.1. Eksik verileri ortalama ile doldurma

In [12]:
# Mode değerini kontrol et ve ona göre doldur
if not df["Diagnosis"].mode().empty:
    df["Diagnosis"] = df["Diagnosis"].fillna(df["Diagnosis"].mode()[0])
else:
    df["Diagnosis"] = df["Diagnosis"].fillna("Benign")  # Varsayılan değer olarak 'Benign' atanabilir.

In [13]:
df.drop(columns=["Diagnosis"], inplace=True)

3. Yeni verisetini oluşturma ve kaydetme

Ön işlenmiş veriyi tekrar yeni bir veri setine çevrildi ve analiz - modelleme için kaydedildi.
Ön işlenmiş veri kümesi yeni bir CSV dosyası olarak kaydedildi.

In [14]:
processed_file_path = "datasets/processed_thyroid_cancer_data.csv"
df.to_csv(processed_file_path, index=False)

print(f"Veri seti başarıyla kaydedildi: {processed_file_path}")

Veri seti başarıyla kaydedildi: datasets/processed_thyroid_cancer_data.csv


4. Veri setini küçük parçaya ayırma (%10)

In [15]:
import pandas as pd

# Ön işleme yapılmış veri setini yükleyelim
file_path = "datasets/new_processed_thyroid_cancer_data.csv"  # Orijinal dosya adı
df = pd.read_csv(file_path)

# Veri setinin %10'unu seçerek daha küçük bir veri seti oluşturalım
df_sampled = df.sample(frac=0.1, random_state=42)

# Yeni dosya adı belirleyelim
small_dataset_path = "datasets/small_processed_thyroid_cancer_data.csv"

# Küçültülmüş veri setini CSV formatında kaydedelim
df_sampled.to_csv(small_dataset_path, index=False)

print(f"Küçültülmüş veri seti kaydedildi: {small_dataset_path}")

Küçültülmüş veri seti kaydedildi: datasets/small_processed_thyroid_cancer_data.csv
